```{=html}
<style type="text/css">

h1.title {
  font-size: 20px;
  color: White;
  text-align: center;
}
h4.author { /* Header 4 - and the author and data headers use this too  */
    font-size: 16px;
  font-family: "Source Sans Pro Semibold", Times, serif;
  color: Red;
  text-align: center;
}
h4.date { /* Header 4 - and the author and data headers use this too  */
  font-size: 16px;
  font-family: "Source Sans Pro Semibold", Times, serif;
  color: Red;
  text-align: center;
}

.justify {
  text-align: justify !important
}

/* Adjust width of the Quarto tab buttons */
.panel-tabset .nav-link {
    width: 250px; /* Adjust the width value to your preference */
}


</style>
```



------------------------------------------------------------------------

:::{.column-page}

::: {style="text-align:center"}
<h2>Image Retrieval by Hashing </h2>
:::


</br>

### Introduction

::: {.justify}
This work develops a compact image hash code learning framework based on the COSFIRE filter banks for efficient similarity search and retrieval. Images are first passed through class-specific COSFIRE filters designed to activate on visually discriminative patterns. These feature vectors are input to a simple multi-layer perceptron (MLP)  network to learn binary hash codes that should capture the semantic similarity of the images, enabling efficient matching of hash codes of database images for fast retrieval. Our experiments on an image dataset demonstrate the potential of this straightforward approach for developing compact hash codes based rotation-invariant COSFIRE descriptors.  

::: {.callout-tip}
MLP is a type of artificial neural network consisting of multiple layers of neurons. The neurons in the MLP typically use nonlinear activation functions, allowing the network to learn complex patterns in data.
:::
:::


### Data description

::: {.justify}
The input data consists of a set of descriptors extracted for each image using COSFIRE filters. Specifically, 50 COSFIRE filters are designed for each image class. When applied to a given image, each COSFIRE filter produces a maximum response value. By concatenating the maximum response across the 50 filters per class, a 200-element descriptor vector is obtained for that image (since there are 4 classes).

This process is applied to every image, resulting in a dataframe where each row contains the 200-element descriptor vector corresponding to an image. So each image is represented by a n-dimensional vector (n=200 in this case) of COSFIRE filter response values, which encode visual characteristics that help differentiate between classes.

::: {.callout-tip}
The dataframe stores these image descriptor vectors, with each row representing a single image and each column representing the maximum response from one of the 200 total COSFIRE filters applied. This serves as the input data capturing image features that will be further transformed into a k-bit hash code for efficient similarity search and retrieval. The compact hash representation helps quickly locate the most similar images from the database given a new query image.
:::

:::


In [ ]:
#$Env:QUARTO_PYTHON = "C:\Users\P307791\Anaconda3\python.exe"

from cosfire_workflow_utils import *
df_training, df_testing, train_label_code, valid_label_code, _ = get_data(r"G:\My Drive\cosfire\COSFIREdescriptor.mat")

dic_labels = { 2: 'Bent',
                3: ' Compact',
                 0: 'FRI',
                 1: 'FRII'
              }

#rename 'label_name' to 'label_code'
df_training.rename(columns={'label_name': 'label_code'}, inplace=True) 
df_testing.rename(columns={'label_name': 'label_code'}, inplace=True)         
df_training['label_name'] = df_training['label_code'].map(dic_labels)
df_testing['label_name'] = df_testing['label_code'].map(dic_labels)

data_preview = pd.concat([df_training.iloc[:, :10], df_training[['label_name']]], axis=1).head(10)
data_preview.columns = ['descrip_1', 'descrip_2', 'descrip_3', 'descrip_4', 'descrip_5',
       'descrip_6', 'descrip_7', 'descrip_8', 'descrip_9', 'descrip_10',
       'galaxy']
df_training_new = pd.concat([df_training,df_testing], ignore_index=True)


df_training.drop('label_name', axis=1, inplace=True)
df_testing.drop('label_name', axis=1, inplace=True)



display(Markdown(data_preview.to_markdown(index = True)))

### Model Training

::: {.justify}

#### Loss function

Let $\Omega$ represent the COSFIRE descriptor embedding space for radio galaxy images. The objective is to discover a mapping function $F : \Omega → {+1, −1}^{k}$ that translates the embedding space to a k-bit binary code space. This mapping should be learned in such a way that visually or semantically similar radio galaxy images are assigned binary codes that are close to each other, while dissimilar images are mapped to binary codes that are far apart in the binary code space. 

\begin{aligned} 
L_r\left(\mathbf{b}_1, \mathbf{b}_2, y\right) & =\frac{1}{2}(1-y)\left\|\mathbf{b}_1-\mathbf{b}_2\right\|_2^2 \\
 & +\frac{1}{2} y \max \left(m-\left\|\mathbf{b}_1-\mathbf{b}_2\right\|_2^2, 0\right) \\ 
 & +\alpha\left(\left\|\left|\mathbf{b}_1\right|-\mathbf{1}\right\|_1+\left\|\left|\mathbf{b}_2\right|-\mathbf{1}\right\|_1\right)
 \end{aligned}

where $D_h(· , ·)$ denotes the Hamming distance between two binary vectors, and m > 0 is a margin threshold parameter.



In this loss function: y = 0 if they are similar, and y = 1 otherwise 

::: {.callout-note}
- The first term encourages similar pairs to have small distances - punishes similar images mapped to different binary codes.

- The second term  encourages dissimilar pairs to have distances greater than the margin m punishes dissimilar images mapped to close binary codes when their Hamming distance falls below the margin threshold m. Only those dissimilar pairs having their distance within a radius are eligible to contribute to the loss function.
:::


Suppose that there are N training pairs randomly selected from the training images ${(I_i,1, I_i,2, y_i)|i = 1, ..., N}$, our goal is to minimize the overall loss function:


\begin{gathered}
\mathcal{L}=\sum_{i=1}^N L\left(\mathbf{b}_{i, 1}, \mathbf{b}_{i, 2}, y_i\right) \\
\text { s.t. } \mathbf{b}_{i, j} \in\{+1,-1\}^k, i \in\{1, \ldots, N\}, j \in\{1,2\}
\end{gathered}





::: {.callout-tip}
Regularization: To reduce the discrepancy between Euclidean space and the Hamming space a commonly used relaxation scheme is to utilize sigmoid or tanh function to approximate the thresholding procedure. A regularizer is applied to help obtain real-valued network outputs (from sigmoid/tanh/relu etc) to approach the desired discrete binary-like values (e.g 0,1).
:::

We train a simple MLP network architecture. By optimizing this loss, the neural network learns to transform the input into a latent feature space that accentuates similarities and differences critical for distinguishing images effectively. The resulting model provides an end-to-end learning pipeline from raw image inputs to a k(36)-bit compact hash code amenable for efficient image retrieval and matching. Our experiments demonstrate the potential of this straightforward architecture and training approach for image hashing.

### Binarization and Mean Average Precision (mAP)

Mean average precision (mAP) is a commonly used evaluation metric in image retrieval tasks. It measures the average precision across all queries in the dataset. Precision is defined as the number of relevant images retrieved divided by the total number of images retrieved.



The formula for MAP is given as:


\begin{equation}
\text{AP} = \frac{1}{\text{GTP} }\sum_{i=1}^{n}\text{Precision}(i)\times\text{Rel}(i),
\end{equation}

\begin{equation}
\text{mAP} = \frac{1}{N_q }\sum_{j=1}^{N_q}AP_j,
\end{equation}


where AP represents the average precision of one query, with $n$ being the total number of reference images, and $\text{GTP}$ the  total number of ground truth positives, $\text{Precision}(i)$ is the precision of the top $i$ ranked reference images and $\text{Rel}(i)$ is an indicator variable that is 1 if the $i$th image is relevant and 0 otherwise. Finally, the mAP is computed as the average of all AP values obtained for all $N_q$ query images.

::: {.callout-tip}

- In image retrieval, a query is typically an image, and the task is to retrieve a set of relevant images from a large dataset. The mAP metric is used to evaluate the performance of the retrieval system by comparing the retrieved images to a set of ground-truth relevant images for each query.

- mAP takes into account both the relevance and the ranking of the retrieved images. A high mAP score indicates that the retrieval system is able to retrieve a high proportion of relevant images, and that these images are ranked highly in the retrieved set.
:::
:::


In [ ]:
class CosfireNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(CosfireNet, self).__init__()
        self.hd = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.Tanh(),
            nn.Dropout(0.3),
            nn.Linear(64, output_size),
            nn.Tanh()
        )

    def forward(self, x):
        return self.hd(x)
    
# Data
class CosfireDataset(Dataset):
    def __init__(self, dataframe):
        self.data = torch.tensor(dataframe.iloc[:, :-1].values, dtype=torch.float32)
        self.labels = torch.tensor(dataframe.iloc[:, -1].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Hyperparameters
input_size = 200
output_size = 24
learning_rate = 0.01
batch_size = 16
alpha = 0.0001
epochs = 600
props = 0.1
margin = 24

train_df, valid_df = train_test_split(df_training, test_size = props, random_state = 42)

train_dataset = CosfireDataset(train_df)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

valid_dataset = CosfireDataset(valid_df)
valid_dataloader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)



# Instantiate CosfireNet
model = CosfireNet(input_size, output_size)

def DSHLoss(u, y, alpha, margin):
    #alpha = 1e-5  # m > 0 is a margin. The margin defines a radius around X1,X2. Dissimilar pairs contribute to the loss
    # function only if their distance is within this radius
    m = 2 * margin  # Initialize U and Y with the current batch's embeddings and labels
    y = y.int()
    # Create a duplicate y_label
    y = y.unsqueeze(1).expand(len(y),len(y))
    y_label = torch.ones_like(torch.empty(len(y), len(y)))
    y_label[y == y.t()] = 0

    dist = torch.cdist(u, u, p=2).pow(2)
    loss1 = 0.5 * (1 - y_label) * dist
    loss2 = 0.5 * y_label * torch.clamp(m - dist, min=0)

    B1 = torch.norm(torch.abs(u) - 1, p=1, dim=1)
    B2 = B1.unsqueeze(1).expand(len(y), len(y))

    loss3 = (B2 + B2.t()) * alpha
    minibatch_loss = torch.mean(loss1 + loss2 + loss3)
    return minibatch_loss


################################################################################



################################################################################


optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.95)

# Lists to store training and validation losses
train_losses = []
val_losses = []

# Variables to keep track of the best model
#best_val_loss = float('inf')

# Training loop


for epoch in tqdm(range(epochs), desc='Training Progress', leave=True):
    model.train()
    total_train_loss = 0.0
    for _, (inputs, labels) in enumerate(train_dataloader):
        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers( i.e, not overwritten) whenever .backward()
        optimizer.zero_grad()
        u = model(inputs)
        loss = DSHLoss(u = u, y=labels, alpha = alpha, margin = margin)
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # Calling the step function on an Optimizer makes an update to its  parameters
        optimizer.step()
        total_train_loss += loss.item()
    #It has been well established that you can achieve increased performance and faster 
    # training on some problems by using a learning rate that changes during training.
    scheduler.step() #optimizer.param_groups[0]['lr']  - updates the lr after every epoch.

    # Calculate average training loss
    average_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(average_train_loss)


    # Validation loop
    model.eval()
    features = []
    total_val_loss = 0.0
    
    with torch.no_grad():
        for val_inputs, val_labels in valid_dataloader:
         
            val_outputs = model(val_inputs)
            features.append(u)
            val_loss = DSHLoss(u = val_outputs, y = val_labels, alpha = alpha, margin = margin)
            total_val_loss += val_loss.item()

    # Calculate average validation loss
    average_val_loss = total_val_loss / len(valid_dataloader)
    val_losses.append(average_val_loss)

    # Save the model if it is the best so far
   #  if average_val_loss < best_val_loss:
   #    best_val_loss = average_val_loss
   #    model_save_path = 'best_model.pth'
   #    torch.save(model, model_save_path)
model_save_path = f'best_model_{output_size}_bit.pth'
torch.save(model, model_save_path)
# Plotting the training and validation loss curves
plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()


In [ ]:
###################################
###       Evaluate              ###
###################################

# Load the best model
model = torch.load(model_save_path)
model.eval()


#################################################################
valid_dataset = CosfireDataset(valid_df)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

train_dataset = CosfireDataset(train_df)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
#################################################################

# Lists to store predictions
predictions = []

# Perform predictions on the testing set
with torch.no_grad():
    for train_inputs, _ in tqdm(train_dataloader, desc='Predicting', leave=True):
        train_outputs = model(train_inputs)
        predictions.append(train_outputs.numpy())

# Flatten the predictions
flat_predictions_train = [item for sublist in predictions for item in sublist]

# Append predictions to the df_testing DataFrame
train_df['predictions'] = flat_predictions_train

#################################################################

# Lists to store predictions
predictions = []

# Perform predictions on the testing set
with torch.no_grad():
    for valid_inputs, _ in tqdm(valid_dataloader, desc='Predicting', leave=True):
        valid_preds = model(valid_inputs)
        predictions.append(valid_preds.numpy())

# Flatten the predictions
flat_predictions_test = [item for sublist in predictions for item in sublist]

# Append predictions to the df_testing DataFrame
valid_df['predictions'] = flat_predictions_test
################################################################


#thresholds = list(range(50,100,0.5))#[30, 50, 55, 65, 70, 85, 90]#)
#thresholds = np.linspace(50, 70, 50).tolist()
# start = 55.714285714
# end = 100
# step = 0.408163265306122*10

# numbers1 = [start + i * step for i in range(int((end - start) / step) + 1)]

# start = 0
# end = 55.714285714
# step = 0.408163265306122*10

# numbers2 =  [start + i * step for i in range(int((end - start) / step) + 1)]


#thresholds = numbers2 + numbers1


thresholds = list(range(0,100,5))#[30, 50, 55, 65, 70, 85, 90]#)
# thresholds = np.linspace(50, 70, 50).tolist()

mAP_results = []
for _,thresh in enumerate(thresholds):

  maP,train_binary, train_label, valid_binary, valid_label = mAP_values(train_df,valid_df,thresh, percentile = True,topk=100)
  mAP_results.append(maP)



data = {'mAP': mAP_results,
        'threshold': thresholds}

df = pd.DataFrame(data)

# Find the index of the maximum mAP value
max_map_index = df['mAP'].idxmax()

# Retrieve the threshold corresponding to the maximum mAP
threshold_max_map = df.loc[max_map_index, 'threshold']

maP,train_binary, train_label, valid_binary, valid_label = mAP_values(train_df,valid_df,thresh = threshold_max_map, percentile = True, topk=100)

# Plot the line curve
plt.plot(thresholds, mAP_results,  linestyle='-',color = 'red')
plt.xlabel('Threshold (Percentile)')
plt.ylabel('mAP')
plt.show()

print('The optimal threshold is: ', threshold_max_map)
print('The Best Validation mAP is: ',maP)

#### Model Testing

Now applying the same model (best model from the validation data) to the test data with the best threshold as per the validation data.


In [ ]:
test_dataset = CosfireDataset(df_testing)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

train_dataset = CosfireDataset(df_training)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)



# %%
# Lists to store predictions
predictions = []

# Perform predictions on the testing set
with torch.no_grad():
    for train_inputs, _ in tqdm(train_dataloader, desc='Predicting', leave=True):
        train_outputs = model(train_inputs)
        predictions.append(train_outputs.numpy())

# Flatten the predictions
flat_predictions_train = [item for sublist in predictions for item in sublist]

# Append predictions to the df_testing DataFrame
df_training['predictions'] = flat_predictions_train

#################################################################

# Lists to store predictions
predictions = []

# Perform predictions on the testing set
with torch.no_grad():
    for test_inputs, _ in tqdm(test_dataloader, desc='Predicting', leave=True):
        test_outputs = model(test_inputs)
        predictions.append(test_outputs.numpy())

# Flatten the predictions
flat_predictions_test = [item for sublist in predictions for item in sublist]

# Append predictions to the df_testing DataFrame
df_testing['predictions'] = flat_predictions_test


maP,train_binary, train_label, test_binary, test_label = mAP_values(df_training,df_testing,thresh = threshold_max_map, percentile = True, topk=100)

print('At the optimal threshold: ', threshold_max_map)
print('The Test mAP is: ',maP)

:::{.column-page}


### Model Predictions overview

::: {.panel-tabset}

####  Test predictions


In [ ]:
dic_labels_rev = { 2:'Bent',
                3:'Compact',
                  0:'FRI',
                  1: 'FRII'
              }

df_training['labels'] = df_training['label_code'].map(dic_labels_rev)
df_testing['labels'] = df_testing['label_code'].map(dic_labels_rev)
# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Iterate over label_code values
for label_code in range(4):
    dff = df_testing.query(f'label_code == {label_code}')
    out_array_train = []
    dd_train = np.array([out_array_train.extend(np.array(out)) for _, out in enumerate(dff.predictions)])
    out_array_train = np.array(out_array_train)
    
    # Plot the KDE curve with a hue
    sns.kdeplot(out_array_train, label=f'{dic_labels[label_code]}', ax=ax)

# Set the x-axis limits
ax.set_xlim(-1, 1)

# Customize the plot
ax.set_xlabel('Value')
ax.set_ylabel('Density')
ax.grid(True)
ax.legend()

# Display the plot
plt.show()
plt.close()

df = pd.DataFrame([df_testing.predictions[0]])
df.columns = ['hash_'+str(i) for i in range(36)]
df['label'] = df_testing.labels[0]
for j in range(1,400,20):
   df2 = pd.DataFrame([df_testing.predictions[j]])
   df2.columns = ['hash_'+str(i) for i in range(36)]
   df2['label'] = df_testing.labels[j]
   df = pd.concat([df,df2])

display(Markdown(df.to_markdown(index = True)))

####  Train predictions


In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Iterate over label_code values
for label_code in range(4):
    dff = df_training.query(f'label_code == {label_code}')
    out_array_train = []
    dd_train = np.array([out_array_train.extend(np.array(out)) for _, out in enumerate(dff.predictions)])
    out_array_train = np.array(out_array_train)
    
    # Plot the KDE curve with a hue
    sns.kdeplot(out_array_train, label=f'{dic_labels[label_code]}', ax=ax)

# Set the x-axis limits
ax.set_xlim(-1, 1)

# Customize the plot
ax.set_xlabel('Value')
ax.set_ylabel('Density')
ax.grid(True)
ax.legend()

# Display the plot
plt.show()
plt.close()


df = pd.DataFrame([df_training.predictions[0]])
df.columns = ['hash_'+str(i) for i in range(36)]
df['label'] = df_training.labels[0]
for j in range(1,1180,50):
   df2 = pd.DataFrame([df_training.predictions[j]])
   df2.columns = ['hash_'+str(i) for i in range(36)]
   df2['label'] = df_training.labels[j]
   df = pd.concat([df,df2])

display(Markdown(df.to_markdown(index = True)))

####  T-SNE projection (Test) 


In [ ]:
# %%
array_dat = []
for i in range(df_testing['predictions'].shape[0]):
  array_dat.append(list(df_testing['predictions'].iloc[i]))

array_dat = np.array(array_dat)
array_dat.shape

y = df_testing.labels
tsne = TSNE(n_components=2, random_state=42)
z = tsne.fit_transform(array_dat)
df = pd.DataFrame()
df["y"] = y
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 4),
                data=df).set(title="Model last layer T-SNE projection")

plt.show()
plt.close()

####  T-SNE projection (Train) 


In [ ]:
# %%
array_dat = []
for i in range(df_training['predictions'].shape[0]):
  array_dat.append(list(df_training['predictions'].iloc[i]))

array_dat = np.array(array_dat)
y = df_training.labels
tsne = TSNE(n_components=2, random_state=42)
z = tsne.fit_transform(array_dat)
df = pd.DataFrame()
df["y"] = y
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 4),
                data=df).set(title="Model last layer T-SNE projection")


plt.show()
plt.close()

####  T-SNE projection (Test & Train) 

In [ ]:
array_dat = []
for i in range(df_training['predictions'].shape[0]):
  array_dat.append(list(df_training['predictions'].iloc[i]))

for i in range(df_testing['predictions'].shape[0]):
  array_dat.append(list(df_testing['predictions'].iloc[i]))
array_dat = np.array(array_dat)


y = np.array(pd.concat([df_training.labels,df_testing.labels]))
tsne = TSNE(n_components=2, random_state=42)
z = tsne.fit_transform(array_dat)
df = pd.DataFrame()
df["y"] = y
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 4),
                data=df).set(title="Model last layer T-SNE projection")
plt.show()
plt.close()

:::




### View the image retrieval - COSFIRE Approach

In [ ]:
dic_labels = { 2: 'Bent',
                3: ' Compact',
                 0: 'FRI',
                 1: 'FRII'
              }

df_training['labels'] = df_training['label_code'].map(dic_labels)
df_testing['labels'] = df_testing['label_code'].map(dic_labels)

df_labels_paths = pd.DataFrame()
df_labels_paths['paths'] = glob.glob('../data/test/*/*')
df_labels_paths['label'] = df_labels_paths['paths'].apply(lambda x: x.split(os.path.sep)[1] )
dic_labels_rev = { 'Bent':2,
                'Compact':3,
                  'FRI':0,
                  'FRII':1
              }
df_labels_paths['label_code'] = df_labels_paths['label'].map(dic_labels_rev)
df_labels_paths = df_labels_paths.sort_values('label_code')
df_labels_paths = df_labels_paths.reset_index()[['paths', 'label', 'label_code']]

df_labels_train_paths = pd.DataFrame()
df_labels_train_paths['paths'] = glob.glob('../data/train/*/*')
df_labels_train_paths['label'] = df_labels_train_paths['paths'].apply(lambda x: x.split(os.path.sep)[1] )
df_labels_train_paths['label_code'] = df_labels_train_paths['label'].map(dic_labels_rev)
df_labels_train_paths = df_labels_train_paths.sort_values('label_code')
df_labels_train_paths = df_labels_train_paths.reset_index()[['paths', 'label', 'label_code']]

def perf_percentages(input_data):
    unique, counts = np.unique(input_data, return_counts=True)
    df = pd.DataFrame()
    df['unique'] = unique
    df['counts'] = counts
    df['Percentage'] = np.round(counts / counts.sum() * 100)
    return df

    
def query_image(test_image_index = 190, 
test_images_paths = df_labels_paths,
train_images_db_paths = df_labels_train_paths,
train_images_db = train_binary,
test_binary = test_binary):

         
    print('Test Image is: ', test_images_paths.label[test_image_index])
    fig = plt.figure(figsize=(3, 3))
    image_test = Image.open(test_images_paths.paths[test_image_index])
    image_test = torch.from_numpy(np.array(image_test))
    plt.imshow(image_test[:, :, 1], cmap='viridis')
    plt.axis('off')
    plt.show()

    test_image = test_binary[test_image_index]  
    #np.count_nonzero(np.array([0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
      # 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0])==np.array([0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
      # 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0]))
    similarity_distance = np.count_nonzero(test_image != train_images_db, axis=1)
    sort_indices = np.argsort(similarity_distance)
    top_indices = sort_indices[:100]
    #print(top_indices)
    paths_to_imgs = [train_images_db_paths.paths[index] for _,index in enumerate(top_indices)]
    df = perf_percentages([train_images_db_paths.label[index] for index in top_indices])
    print(df)
    cols = 8
    rows = 4

    fig = plt.figure(figsize=(2 * cols, 2 * rows))
    for col in range(cols):
        for i, img_path in enumerate(paths_to_imgs[:32]):
            ax = fig.add_subplot(rows, cols, i + 1)
            ax.grid(visible=False)
            ax.axis("off")
            image = Image.open(img_path)
            image = torch.from_numpy(np.array(image))
            ax.imshow(image[:, :, 1], cmap='viridis')
            ax.set_title(img_path.split(os.path.sep)[1])

    plt.show()
    


#df_testing.labels.value_counts(sort=False)

::: {.panel-tabset}

#### FRI 


In [ ]:
#0-200

#query_image(test_image_index = 5)


</br>


In [ ]:
query_image(test_image_index = random.randint(0, 100))

#### FRII


In [ ]:
#101-200
#query_image(test_image_index =105)


</br>


In [ ]:
query_image(test_image_index = random.randint(101, 202))

#### Bent


In [ ]:
#200 - 300
#query_image(test_image_index =250)

</br>


In [ ]:
query_image(test_image_index = random.randint(205, 300))

#### Compact


In [ ]:
#300-400
#query_image(test_image_index =403)

</br>


In [ ]:
query_image(test_image_index = random.randint(310, 400))

:::



</br>


:::
:::